 **Load Libraries**

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import nltk
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import DataLoader
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report,
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay
)
from sklearn.utils import resample
from torch.optim import AdamW
import warnings

warnings.filterwarnings("ignore")

 **Environment Setup**

In [ ]:
np.random.seed(42)
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🖥️ Using device:", device)

nltk.download("stopwords")
nltk.download("wordnet")

 **Load and Prepare Dataset**

In [ ]:
file_path = r"C:\Users\VARSHINI.M\OneDrive\Desktop\7th sem\interdisclipinary\Datasets\Sentiment Analysis on Financial Tweets\tweet_sentiment.csv"
df = pd.read_csv(file_path, encoding="latin-1", header=None)
df.columns = ["text", "sentiment"]


**Clean and filter sentiment values**

In [ ]:
df = df[df["sentiment"].isin(["positive", "neutral", "negative", "1", "0", "-1"])]
sentiment_map = {"negative": 0, "neutral": 1, "positive": 2, "-1": 0, "0": 1, "1": 2}
df["sentiment"] = pd.to_numeric(df["sentiment"].replace(sentiment_map), errors="coerce")
df.dropna(subset=["sentiment"], inplace=True)
df["sentiment"] = df["sentiment"].astype(int)
print(" Original Class Distribution:")
print(df["sentiment"].value_counts(), "\n")


**Text Preprocessing**

In [ ]:
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    text = re.sub(r"@\w+|#", "", text)
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    tokens = [lemmatizer.lemmatize(w) for w in text.split() if w not in stop_words]
    return " ".join(tokens)

df["clean_text"] = df["text"].astype(str).apply(clean_text)

**Handle Class Imbalance**

In [ ]:
counts = df["sentiment"].value_counts()
if len(counts) > 1 and counts.max() / counts.min() > 1.5:
    print("⚖️ Oversampling minority classes...")
    df_neg = df[df["sentiment"] == 0]
    df_neu = df[df["sentiment"] == 1]
    df_pos = df[df["sentiment"] == 2]
    max_size = max(len(df_neg), len(df_neu), len(df_pos))

    df_neg_over = resample(df_neg, replace=True, n_samples=max_size, random_state=42)
    df_neu_over = resample(df_neu, replace=True, n_samples=max_size, random_state=42)
    df_pos_over = resample(df_pos, replace=True, n_samples=max_size, random_state=42)

    df = pd.concat([df_neg_over, df_neu_over, df_pos_over])
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
else:
    print(" Class distribution is already balanced or uniform.")

print(" Final Class Distribution:")
print(df["sentiment"].value_counts(), "\n")

**Split Dataset**

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(
    df["clean_text"], df["sentiment"], test_size=0.3, stratify=df["sentiment"], random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

print(f"📚 Train: {len(X_train)} | Val: {len(X_val)} | Test: {len(X_test)}\n")

**Tokenization using FinBERT**

In [ ]:
tokenizer = BertTokenizer.from_pretrained("ProsusAI/finbert")

def encode_texts(texts, max_len=128):
    return tokenizer(
        texts.tolist(),
        padding=True,
        truncation=True,
        max_length=max_len,
        return_tensors="pt"
    )

train_enc = encode_texts(X_train)
val_enc = encode_texts(X_val)
test_enc = encode_texts(X_test)

**Dataset Wrapper**

In [ ]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": torch.tensor(self.labels.iloc[idx])
        }

train_ds = SentimentDataset(train_enc, y_train)
val_ds = SentimentDataset(val_enc, y_val)
test_ds = SentimentDataset(test_enc, y_test)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=16, shuffle=False)

**FinBERT + LSTM Model**

In [ ]:
class FinBERT_LSTM(nn.Module):
    def __init__(self, hidden_dim=128, num_classes=3):
        super(FinBERT_LSTM, self).__init__()
        self.bert = BertModel.from_pretrained("ProsusAI/finbert")
        self.lstm = nn.LSTM(
            input_size=self.bert.config.hidden_size,
            hidden_size=hidden_dim,
            batch_first=True,
            bidirectional=True
        )
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():  # Freeze FinBERT
            bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        lstm_out, _ = self.lstm(bert_output.last_hidden_state)
        pooled = torch.mean(lstm_out, dim=1)
        out = self.dropout(pooled)
        return self.fc(out)

model = FinBERT_LSTM().to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

**Training Loop**

In [ ]:
def evaluate(loader):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for batch in loader:
            ids = batch["input_ids"].to(device)
            mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(ids, mask)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total, total_loss / len(loader)

EPOCHS = 5
for epoch in range(EPOCHS):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        optimizer.zero_grad()
        ids = batch["input_ids"].to(device)
        mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(ids, mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    train_loss = total_loss / len(train_loader)
    val_acc, val_loss = evaluate(val_loader)
    test_acc, test_loss = evaluate(test_loader)

    print(f"\n Epoch {epoch+1}/{EPOCHS}")
    print(f"Train → Acc: {train_acc*100:.2f}% | Loss: {train_loss:.4f}")
    print(f"Val   → Acc: {val_acc*100:.2f}% | Loss: {val_loss:.4f}")
    print(f"Test  → Acc: {test_acc*100:.2f}% | Loss: {test_loss:.4f}")
    print("-"*60)

**Plot Accuracy and Loss Curves**

In [ ]:
epochs_range = range(1, EPOCHS + 1)

plt.figure(figsize=(14, 6))

# Accuracy Plot
plt.subplot(1, 2, 1)
plt.plot(epochs_range, history["train_acc"], marker='o', label="Train Accuracy")
plt.plot(epochs_range, history["val_acc"], marker='o', label="Validation Accuracy")
plt.plot(epochs_range, history["test_acc"], marker='o', label="Test Accuracy")
plt.title("📈 Model Accuracy (Train / Val / Test)")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)

# Loss Plot
plt.subplot(1, 2, 2)
plt.plot(epochs_range, history["train_loss"], marker='o', label="Train Loss")
plt.plot(epochs_range, history["val_loss"], marker='o', label="Validation Loss")
plt.plot(epochs_range, history["test_loss"], marker='o', label="Test Loss")
plt.title("📉 Model Loss (Train / Val / Test)")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

**Save and Reload Model**

In [ ]:
MODEL_PATH = "finbert_lstm_model.pth"
torch.save(model.state_dict(), MODEL_PATH)
print(f" Model saved as {MODEL_PATH}\n")

model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

**Prediction Function**

In [ ]:
def predict_sentiment(sentence):
    cleaned = clean_text(sentence)
    encoded = tokenizer(cleaned, return_tensors="pt", truncation=True, padding=True, max_length=128)
    input_ids = encoded["input_ids"].to(device)
    attn_mask = encoded["attention_mask"].to(device)
    with torch.no_grad():
        logits = model(input_ids, attn_mask)
        pred = torch.argmax(logits, dim=1).item()
    label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}
    return label_map[pred]

# Example Predictions (2-line sentences)
print("\n💬 Example Predictions:")
examples = [
    "The company reported record profits this quarter! Sales exceeded expectations across all regions.",
    "The stock market is unstable due to new regulations. Many investors are pulling back from risky assets.",
    "Investors are unsure about the future of this firm. Recent management changes raised serious concerns.",
    "The merger between the two banks was well received by analysts. Shares rose sharply after the announcement.",
    "Weak consumer demand continues to hurt quarterly earnings. The firm announced potential layoffs next month.",
    "The government’s new tax incentives boosted corporate confidence. Economic growth projections were revised upward.",
    "Rising inflation has reduced purchasing power. Retail stocks dropped amid declining customer sentiment.",
    "The company’s innovative AI product attracted major investors. Market experts predict strong revenue growth."
]
for s in examples:
    print(f"{s} → {predict_sentiment(s)}")


**Confusion Matrix**

In [ ]:
def plot_confusion_matrix_all(model, datasets, true_labels, set_names, normalize=False):
    model.eval()
    num_sets = len(datasets)
    fig, axes = plt.subplots(1, num_sets, figsize=(6*num_sets, 5))
    if num_sets == 1:
        axes = [axes]

    for i, (dataset, y_true, name) in enumerate(zip(datasets, true_labels, set_names)):
        loader = DataLoader(dataset, batch_size=32)
        preds = []
        with torch.no_grad():
            for batch in loader:
                ids = batch["input_ids"].to(device)
                mask = batch["attention_mask"].to(device)
                outputs = model(ids, mask)
                preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())

        cm = confusion_matrix(y_true, preds, normalize="true" if normalize else None)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Negative", "Neutral", "Positive"])
        disp.plot(ax=axes[i], cmap="Blues", values_format=".2f" if normalize else "d", colorbar=False)
        axes[i].set_title(f"{name} Confusion Matrix")

    plt.suptitle("📉 Confusion Matrices - FinBERT + LSTM", fontsize=14, y=1.03)
    plt.tight_layout()
    plt.show()

plot_confusion_matrix_all(
    model,
    datasets=[train_ds, val_ds, test_ds],
    true_labels=[y_train, y_val, y_test],
    set_names=["Train", "Validation", "Test"],
    normalize=True
)

**Overall Confusion Matrix**

In [ ]:
def plot_overall_confusion_matrix(model, datasets, true_labels, normalize=False):
    model.eval()
    all_preds, all_true = [], []
    for dataset, y_true in zip(datasets, true_labels):
        loader = DataLoader(dataset, batch_size=32)
        preds = []
        with torch.no_grad():
            for batch in loader:
                ids = batch["input_ids"].to(device)
                mask = batch["attention_mask"].to(device)
                outputs = model(ids, mask)
                preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
        all_preds.extend(preds)
        all_true.extend(y_true)

    cm = confusion_matrix(all_true, all_preds, normalize="true" if normalize else None)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Negative", "Neutral", "Positive"])
    disp.plot(cmap="Blues", values_format=".2f" if normalize else "d")
    plt.title("📉 Overall Confusion Matrix - FinBERT + LSTM")
    plt.tight_layout()
    plt.show()

    print("\n📊 Classification Report (Overall):")
    print(classification_report(all_true, all_preds, target_names=["Negative", "Neutral", "Positive"]))

plot_overall_confusion_matrix(
    model,
    datasets=[train_ds, val_ds, test_ds],
    true_labels=[y_train, y_val, y_test],
    normalize=True
)

 **LOAD SAVED MODEL & PREDICT IN REAL-TIME**

In [ ]:
MODEL_PATH = "finbert_lstm_balanced_model.pth"
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.to(device)
model.eval()
print(f"Loaded model from: {MODEL_PATH}")

#  Define Prediction Function
def predict_sentiment(sentence):
    model.eval()
    cleaned = clean_text(sentence)
    encoded = tokenizer(cleaned, return_tensors='pt', truncation=True, padding=True, max_length=128)
    input_ids = encoded['input_ids'].to(device)
    attn_mask = encoded['attention_mask'].to(device)
    with torch.no_grad():
        logits = model(input_ids, attn_mask)
        pred = torch.argmax(logits, dim=1).item()
    sentiment_val = idx_to_sentiment[pred]
    return f"{idx_to_label[pred]} ({sentiment_val})"

# Example Predictions
print("\n💬 Example Predictions:")
examples = [
    "The company reported record profits this quarter! Sales exceeded expectations across all regions.",
    "The stock market is unstable due to new regulations. Many investors are pulling back from risky assets.",
    "Investors are unsure about the future of this firm. Its recent management changes raised serious concerns.",
    "The merger between the two banks was well received by analysts. Shares rose sharply after the announcement.",
    "Weak consumer demand continues to hurt quarterly earnings. The firm announced potential layoffs next month.",
    "The government’s new tax incentives boosted corporate confidence. Economic growth projections were revised upward.",
    "Rising inflation has reduced purchasing power. Retail stocks dropped amid declining customer sentiment.",
    "The company’s innovative AI product attracted major investors. Market experts predict strong revenue growth."
]
for s in examples:
    print(f"{s} → {predict_sentiment(s)}")


# **Modelwithout pre trained model(Finbert)**

**Import libraries**

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from tqdm import tqdm
from collections import Counter
import warnings
import os
warnings.filterwarnings("ignore")


**Environment Setup**

In [ ]:
np.random.seed(42)
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

nltk.download('stopwords')
nltk.download('wordnet')

**Load Dataset**

In [ ]:
file_path = "/content/tweet_sentiment.csv"  #
    df = pd.read_csv(file_path, encoding='latin-1', header=None)
    df.columns = ['text', 'sentiment']

    df = df[df['sentiment'].isin(['positive', 'negative', 'neutral', '1', '0', '-1'])]
    if df.empty:
        raise ValueError("DataFrame is empty after filtering. Check the CSV format.")

    sentiment_map = {'negative': 0, 'neutral': 1, 'positive': 2, '-1': 0, '0': 1, '1': 2}
    df['sentiment'] = pd.to_numeric(df['sentiment'].replace(sentiment_map), errors='coerce')
    df = df.dropna(subset=['sentiment'])
    df['sentiment'] = df['sentiment'].astype(int)

    print("Original Class Distribution:")
    print(df['sentiment'].value_counts(), "\n")


**Preprocessing**

In [ ]:
 stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()

    def clean_text(text):
        text = text.lower()
        text = re.sub(r"http\S+|www\S+|https\S+", "", text)
        text = re.sub(r"@\w+|#", "", text)
        text = re.sub(r"[^a-zA-Z\s]", "", text)
        text = re.sub(r"\s+", " ", text).strip()
        tokens = [lemmatizer.lemmatize(w) for w in text.split() if w not in stop_words]
        return " ".join(tokens)

    df['clean_text'] = df['text'].astype(str).apply(clean_text)

**Handle Class Imbalance**

In [ ]:
counts = df['sentiment'].value_counts()
    if counts.max() / counts.min() > 1.5:
        print("Oversampling minority classes...")
        df_neg = df[df['sentiment'] == 0]
        df_neu = df[df['sentiment'] == 1]
        df_pos = df[df['sentiment'] == 2]
        max_size = max(len(df_neg), len(df_neu), len(df_pos))
        df_neg_over = resample(df_neg, replace=True, n_samples=max_size, random_state=42)
        df_neu_over = resample(df_neu, replace=True, n_samples=max_size, random_state=42)
        df_pos_over = resample(df_pos, replace=True, n_samples=max_size, random_state=42)
        df = pd.concat([df_neg_over, df_neu_over, df_pos_over]).sample(frac=1, random_state=42).reset_index(drop=True)

    print("Final Class Distribution:")
    print(df['sentiment'].value_counts(), "\n")

**Split Data (70 : 15 : 15)**

In [ ]:
 X_train, X_temp, y_train, y_temp = train_test_split(
        df['clean_text'], df['sentiment'], test_size=0.3,
        stratify=df['sentiment'], random_state=42
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5,
        stratify=y_temp, random_state=42
    )
    print(f"Train: {len(X_train)} | Val: {len(X_val)} | Test: {len(X_test)}\n")

**Tokenization + Vocabulary**

In [ ]:
def tokenize(text):
        return text.split()

    vocab = Counter()
    for txt in X_train:
        vocab.update(tokenize(txt))
    vocab = {word: idx + 2 for idx, (word, _) in enumerate(vocab.most_common())}
    vocab["<PAD>"] = 0
    vocab["<UNK>"] = 1

    def encode(text, max_len=50):
        tokens = tokenize(text)
        ids = [vocab.get(t, vocab["<UNK>"]) for t in tokens]
        if len(ids) < max_len:
            ids += [vocab["<PAD>"]] * (max_len - len(ids))
        else:
            ids = ids[:max_len]
        return torch.tensor(ids)

    class TextDataset(Dataset):
        def __init__(self, texts, labels):
            self.texts = texts
            self.labels = labels
        def __len__(self):
            return len(self.labels)
        def __getitem__(self, idx):
            x = encode(self.texts.iloc[idx])
            y = torch.tensor(self.labels.iloc[idx])
            return x, y

    train_ds = TextDataset(X_train, y_train)
    val_ds = TextDataset(X_val, y_val)
    test_ds = TextDataset(X_test, y_test)

    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=32)
    test_loader = DataLoader(test_ds, batch_size=32)

**LSTM Model**

In [ ]:
class SimpleLSTM(nn.Module):
        def __init__(self, vocab_size, embed_dim=128, hidden_dim=128, num_classes=3):
            super(SimpleLSTM, self).__init__()
            self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
            self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=True)
            self.dropout = nn.Dropout(0.3)
            self.fc = nn.Linear(hidden_dim * 2, num_classes)

        def forward(self, x):
            emb = self.embedding(x)
            lstm_out, _ = self.lstm(emb)
            pooled = torch.mean(lstm_out, dim=1)
            out = self.dropout(pooled)
            return self.fc(out)

    model = SimpleLSTM(len(vocab), embed_dim=128, hidden_dim=128, num_classes=3).to(device)
    optimizer = AdamW(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()

**Training Loop**

In [ ]:
 def evaluate(loader):
        model.eval()
        total_loss, correct, total = 0, 0, 0
        with torch.no_grad():
            for x, y in loader:
                x, y = x.to(device), y.to(device)
                outputs = model(x)
                loss = criterion(outputs, y)
                total_loss += loss.item()
                preds = torch.argmax(outputs, dim=1)
                correct += (preds == y).sum().item()
                total += y.size(0)
        return correct / total, total_loss / len(loader)

    EPOCHS = 5
    for epoch in range(EPOCHS):
        model.train()
        total_loss, correct, total = 0, 0, 0
        for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
        train_acc = correct / total
        train_loss = total_loss / len(train_loader)
        val_acc, val_loss = evaluate(val_loader)
        test_acc, test_loss = evaluate(test_loader)

        print(f"\nEpoch {epoch+1}/{EPOCHS}")
        print(f"Train → Acc: {train_acc*100:.2f}% | Loss: {train_loss:.4f}")
        print(f"Val   → Acc: {val_acc*100:.2f}% | Loss: {val_loss:.4f}")
        print(f"Test  → Acc: {test_acc*100:.2f}% | Loss: {test_loss:.4f}")
        print("-"*60)


**Plot Accuracy and Loss Curves**

In [ ]:
epochs_range = range(1, EPOCHS + 1)

plt.figure(figsize=(14, 6))

# Accuracy Plot
plt.subplot(1, 2, 1)
plt.plot(epochs_range, history["train_acc"], marker='o', label="Train Accuracy")
plt.plot(epochs_range, history["val_acc"], marker='o', label="Validation Accuracy")
plt.plot(epochs_range, history["test_acc"], marker='o', label="Test Accuracy")
plt.title("📈 Model Accuracy (Train / Val / Test)")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)

# Loss Plot
plt.subplot(1, 2, 2)
plt.plot(epochs_range, history["train_loss"], marker='o', label="Train Loss")
plt.plot(epochs_range, history["val_loss"], marker='o', label="Validation Loss")
plt.plot(epochs_range, history["test_loss"], marker='o', label="Test Loss")
plt.title("📉 Model Loss (Train / Val / Test)")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

**Confusion matrix**

In [ ]:
def plot_confusion_matrix_all(model, datasets, true_labels, set_names, normalize=False):
    model.eval()
    num_sets = len(datasets)
    fig, axes = plt.subplots(1, num_sets, figsize=(6*num_sets, 5))
    if num_sets == 1:
        axes = [axes]

    for i, (dataset, y_true, name) in enumerate(zip(datasets, true_labels, set_names)):
        loader = DataLoader(dataset, batch_size=32)
        preds = []
        with torch.no_grad():
            for batch in loader:
                ids = batch["input_ids"].to(device)
                mask = batch["attention_mask"].to(device)
                outputs = model(ids, mask)
                preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())

        cm = confusion_matrix(y_true, preds, normalize="true" if normalize else None)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Negative", "Neutral", "Positive"])
        disp.plot(ax=axes[i], cmap="Blues", values_format=".2f" if normalize else "d", colorbar=False)
        axes[i].set_title(f"{name} Confusion Matrix")

    plt.suptitle("📉 Confusion Matrices - FinBERT + LSTM", fontsize=14, y=1.03)
    plt.tight_layout()
    plt.show()

plot_confusion_matrix_all(
    model,
    datasets=[train_ds, val_ds, test_ds],
    true_labels=[y_train, y_val, y_test],
    set_names=["Train", "Validation", "Test"],
    normalize=True
)

**Overall Confusion matrix**

In [ ]:
def plot_overall_confusion_matrix(model, datasets, true_labels, normalize=False):
    model.eval()
    all_preds, all_true = [], []
    for dataset, y_true in zip(datasets, true_labels):
        loader = DataLoader(dataset, batch_size=32)
        preds = []
        with torch.no_grad():
            for batch in loader:
                ids = batch["input_ids"].to(device)
                mask = batch["attention_mask"].to(device)
                outputs = model(ids, mask)
                preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
        all_preds.extend(preds)
        all_true.extend(y_true)

    cm = confusion_matrix(all_true, all_preds, normalize="true" if normalize else None)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Negative", "Neutral", "Positive"])
    disp.plot(cmap="Blues", values_format=".2f" if normalize else "d")
    plt.title("📉 Overall Confusion Matrix - FinBERT + LSTM")
    plt.tight_layout()
    plt.show()

    print("\n📊 Classification Report (Overall):")
    print(classification_report(all_true, all_preds, target_names=["Negative", "Neutral", "Positive"]))

plot_overall_confusion_matrix(
    model,
    datasets=[train_ds, val_ds, test_ds],
    true_labels=[y_train, y_val, y_test],
    normalize=True
)

**Save Model**

In [ ]:
torch.save(model.state_dict(), "lstm_sentiment_model.pth")
    print("Model saved as lstm_sentiment_model.pth\n")

**Load Model and Predict**

In [ ]:
def predict_sentiment(sentence):
    cleaned = clean_text(sentence)
    encoded = tokenizer(cleaned, return_tensors="pt", truncation=True, padding=True, max_length=128)
    input_ids = encoded["input_ids"].to(device)
    attn_mask = encoded["attention_mask"].to(device)
    with torch.no_grad():
        logits = model(input_ids, attn_mask)
        pred = torch.argmax(logits, dim=1).item()
    label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}
    return label_map[pred]

# Example Predictions (2-line sentences)
print("\n💬 Example Predictions:")
examples = [
    "The company reported record profits this quarter! Sales exceeded expectations across all regions.",
    "The stock market is unstable due to new regulations. Many investors are pulling back from risky assets.",
    "Investors are unsure about the future of this firm. Recent management changes raised serious concerns.",
    "The merger between the two banks was well received by analysts. Shares rose sharply after the announcement.",
    "Weak consumer demand continues to hurt quarterly earnings. The firm announced potential layoffs next month.",
    "The government’s new tax incentives boosted corporate confidence. Economic growth projections were revised upward.",
    "Rising inflation has reduced purchasing power. Retail stocks dropped amid declining customer sentiment.",
    "The company’s innovative AI product attracted major investors. Market experts predict strong revenue growth."
]
for s in examples:
    print(f"{s} → {predict_sentiment(s)}")


**Load the trained model with Real-time Prediction**

In [ ]:
MODEL_PATH = "finbert_lstm_balanced_model.pth"
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.to(device)
model.eval()
print(f"Loaded model from: {MODEL_PATH}")

#  Define Prediction Function
def predict_sentiment(sentence):
    model.eval()
    cleaned = clean_text(sentence)
    encoded = tokenizer(cleaned, return_tensors='pt', truncation=True, padding=True, max_length=128)
    input_ids = encoded['input_ids'].to(device)
    attn_mask = encoded['attention_mask'].to(device)
    with torch.no_grad():
        logits = model(input_ids, attn_mask)
        pred = torch.argmax(logits, dim=1).item()
    sentiment_val = idx_to_sentiment[pred]
    return f"{idx_to_label[pred]} ({sentiment_val})"

# Example Predictions
print("\n💬 Example Predictions:")
examples = [
    "The company reported record profits this quarter! Sales exceeded expectations across all regions.",
    "The stock market is unstable due to new regulations. Many investors are pulling back from risky assets.",
    "Investors are unsure about the future of this firm. Its recent management changes raised serious concerns.",
    "The merger between the two banks was well received by analysts. Shares rose sharply after the announcement.",
    "Weak consumer demand continues to hurt quarterly earnings. The firm announced potential layoffs next month.",
    "The government’s new tax incentives boosted corporate confidence. Economic growth projections were revised upward.",
    "Rising inflation has reduced purchasing power. Retail stocks dropped amid declining customer sentiment.",
    "The company’s innovative AI product attracted major investors. Market experts predict strong revenue growth."
]
for s in examples:
    print(f"{s} → {predict_sentiment(s)}")
